In [ ]:
# 선생님 코드

# train : 2000장
# val : 1000장
# test : 22장

In [ ]:
# 데이터 경로 지정
train_dir = '/content/drive/MyDrive/Colab Notebooks/사물지능_딥러닝_2022/data/dogs_vs_cats_small/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/사물지능_딥러닝_2022/data/dogs_vs_cats_small/validation'
test_dir ='/content/drive/MyDrive/Colab Notebooks/사물지능_딥러닝_2022/data/dogs_vs_cats_small/test'

In [ ]:
import os
print(len(os.listdir(train_dir)))
print(len(os.listdir(val_dir)))
print(len(os.listdir(test_dir)))

2
2
2


In [ ]:
# 하나의 변수에 이미지파일 전부다 합치기
# 픽셀값 0~255 > 0~1
# 이미지 크기 동일하게 만들어주기 (150,150)
# 라벨링
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 픽셀값 변환
train_gen = ImageDataGenerator(rescale = 1./255)
test_gen = ImageDataGenerator(rescale = 1./255)
val_gen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# flow_from_directory()
train_generator = train_gen.flow_from_directory(
    train_dir, # train 이미지 경로    
    target_size = (150,150), # 변환할 이미지의 크기
    batch_size = 100, # 한번에 변환할 이미지 갯수
    class_mode = 'binary' # 라벨링 진행, categorical : 다중분류
    # 폴더별로 라벨링을 진행 cats(0) , dogs(1)
)

Found 2000 images belonging to 2 classes.


In [ ]:
val_generator = val_gen.flow_from_directory(
    val_dir, # train 이미지 경로    
    target_size = (150,150), # 변환할 이미지의 크기
    batch_size = 100, # 한번에 변환할 이미지 갯수
    class_mode = 'binary' # 라벨링 진행, categorical : 다중분류
    # 폴더별로 라벨링을 진행 cats(0) , dogs(1)
)

Found 1000 images belonging to 2 classes.


In [ ]:
test_generator = test_gen.flow_from_directory(
    test_dir, # train 이미지 경로    
    target_size = (150,150), # 변환할 이미지의 크기
    batch_size = 100, # 한번에 변환할 이미지 갯수
    class_mode = 'binary' # 라벨링 진행, categorical : 다중분류
    # 폴더별로 라벨링을 진행 cats(0) , dogs(1)
)

Found 22 images belonging to 2 classes.


In [ ]:
# 라벨링 결과 확인
print(train_generator.class_indices)

{'cats': 0, 'dogs': 1}


In [ ]:
# CNN 모델 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [ ]:
# 딥러닝 모델 생성
model1 = Sequential()

# 입력층, 특징추출부
model1.add(Conv2D(
    filters = 32, # 특징의 갯수
    kernel_size = (3,3), # 특징의 크기
    input_shape = (150,150,3), # 입력 데이터의 모양
    padding = 'same', # 입력이미지와 출력이미지의 크기를 동일하게
    activation = 'relu'
))

model1.add(MaxPool2D(
    pool_size = (2,2)
))

model1.add(Conv2D(
    filters = 64, # 특징의 갯수
    kernel_size = (3,3), # 특징의 크기
    input_shape = (150,150,3), # 입력 데이터의 모양
    padding = 'same', # 입력이미지와 출력이미지의 크기를 동일하게
    activation = 'relu'
))

model1.add(MaxPool2D(
    pool_size = (2,2)
))
################특징 추출부 끝###############
model1.add(Flatten())
################분류부 시작#################
# 위에서 특징을 추출했기때문에 층을 깊게 쌓지 않아도됨
model1.add(Dense(units = 256, activation = 'relu'))

# 출력층
model1.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
model1.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
model1.fit_generator(
    generator = train_generator,
    epochs = 20,
    validation_data = val_generator
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/20
20/20 [==============================] - 1030s 52s/step - loss: 1.6378 - accuracy: 0.4965 - val_loss: 0.6953 - val_accuracy: 0.4810
Epoch 2/20
20/20 [==============================] - 12s 602ms/step - loss: 0.6883 - accuracy: 0.5405 - val_loss: 0.6904 - val_accuracy: 0.5310
Epoch 3/20
20/20 [==============================] - 12s 591ms/step - loss: 0.6499 - accuracy: 0.6355 - val_loss: 0.6361 - val_accuracy: 0.6110
Epoch 4/20
20/20 [==============================] - 12s 599ms/step - loss: 0.6152 - accuracy: 0.6650 - val_loss: 0.6215 - val_accuracy: 0.6580
Epoch 5/20
20/20 [==============================] - 12s 598ms/step - loss: 0.5068 - accuracy: 0.7700 - val_loss: 0.5998 - val_accuracy: 0.6750
Epoch 6/20
20/20 [==============================] - 13s 652ms/step - loss: 0.4210 - accuracy: 0.8095 - val_loss: 0.5879 - val_accuracy: 0.6930
Epoch 7/20
20/20 [==============================] - 12s 601ms/step - loss: 0.3216 - accuracy: 0.8770 - val_loss: 0.6788 - val_accuracy: 0.6950

In [ ]:
# 과대적합이 관찰되었음
# 학습데이터 증가 2000 > 3000
# 딥러닝 모델 재설계 > dropout()
# 학습데이터 augmentation(확장) : 기존의 이미지에 변화를 줘서 이미지를 사용

In [ ]:
# 이미지 확장(증식)
train_aug = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    horizontal_flip = True,
    fill_mode = 'nearest' # 비어있는 부분을 가장 가까이에 있는 값으로 채움
)

train_aug_generator = train_aug.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 100,
    class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.


In [ ]:
# CNN 모델 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

# 딥러닝 모델 생성
model1 = Sequential()

# 입력층, 특징추출부
model1.add(Conv2D(
    filters = 32, # 특징의 갯수
    kernel_size = (3,3), # 특징의 크기
    input_shape = (150,150,3), # 입력 데이터의 모양
    padding = 'same', # 입력이미지와 출력이미지의 크기를 동일하게
    activation = 'relu'
))

model1.add(MaxPool2D(
    pool_size = (2,2)
))

model1.add(Conv2D(
    filters = 64, # 특징의 갯수
    kernel_size = (3,3), # 특징의 크기
    input_shape = (150,150,3), # 입력 데이터의 모양
    padding = 'same', # 입력이미지와 출력이미지의 크기를 동일하게
    activation = 'relu'
))

model1.add(MaxPool2D(
    pool_size = (2,2)
))

################특징 추출부 끝###############
model1.add(Flatten())
################분류부 시작#################
# 위에서 특징을 추출했기때문에 층을 깊게 쌓지 않아도됨
model1.add(Dense(units = 256, activation = 'relu'))

model1.add(Dropout(0.4))

# 출력층
model1.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
model1.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
model1.fit_generator(
    generator = train_aug_generator,
    epochs = 20,
    validation_data = val_generator
)

In [ ]:
# 전이학습
# 누군가가 만들어놓은 모델 가져오기
# 특성추출방식
from tensorflow.keras.applications import VGG16

conv_base = VGG16(
    weights = 'imagenet', # imagenet에서 사용한 가중치 가져오기
    include_top = False, # 분류기를 사용할것인가? 
    input_shape = (150,150,3)
)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
conv_base.summary()

In [ ]:
# 동결
# 가중치가 갱신되는것을 막는것

# 학습 가능한 가중치의 갯수 확인
len(conv_base.trainable_weights)

26

In [ ]:
# VGG16 전체 층에대해 동결
conv_base.trainable = False

In [ ]:
# 학습 가능한 가중치의 갯수 확인
len(conv_base.trainable_weights)

0

In [ ]:
# 동결된 conv를 사용해서 모델 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model2 = Sequential()

model2.add(conv_base) # 전이학습할 모델의 특성추출부 사용하기

model2.add(Flatten())

model2.add(Dense(units = 256, activation = 'relu'))


# 출력층
model2.add(Dense(units = 1, activation = 'sigmoid'))


model2.summary()

In [ ]:
model2.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy']
               )

In [ ]:
model2.fit_generator(
    generator = train_aug_generator,
    epochs = 20,
    validation_data = val_generator
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/20
20/20 [==============================] - 781s 39s/step - loss: 0.9818 - accuracy: 0.6070 - val_loss: 0.4286 - val_accuracy: 0.8240
Epoch 2/20
20/20 [==============================] - 20s 1s/step - loss: 0.3910 - accuracy: 0.8315 - val_loss: 0.3040 - val_accuracy: 0.8950
Epoch 3/20
20/20 [==============================] - 20s 1s/step - loss: 0.3239 - accuracy: 0.8645 - val_loss: 0.2670 - val_accuracy: 0.8940
Epoch 4/20
20/20 [==============================] - 20s 1s/step - loss: 0.2850 - accuracy: 0.8810 - val_loss: 0.2485 - val_accuracy: 0.9010
Epoch 5/20
20/20 [==============================] - 20s 1s/step - loss: 0.2543 - accuracy: 0.8975 - val_loss: 0.2442 - val_accuracy: 0.9040
Epoch 6/20
20/20 [==============================] - 21s 1s/step - loss: 0.2498 - accuracy: 0.9015 - val_loss: 0.2387 - val_accuracy: 0.9050
Epoch 7/20
20/20 [==============================] - 20s 1s/step - loss: 0.2281 - accuracy: 0.9100 - val_loss: 0.2293 - val_accuracy: 0.9080
Epoch 8/20
20/20 [

In [ ]:
# 미세조정방식

In [ ]:
# model2에서 학습가능한 가중치의 갯수
len(model2.trainable_weights)-

4

In [ ]:
conv_base.summary()

In [ ]:
conv_base = VGG16(
    weights = 'imagenet', # imagenet에서 사용한 가중치 가져오기
    include_top = False, # 분류기를 사용할것인가? 
    input_shape = (150,150,3)
)

In [ ]:
# block5_conv1 ~ 3 층을 학습시켜보자

# block5_conv1 층부터 하위의 층의 동결을 해제
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block4_conv1':
    set_trainable = True
  
  if set_trainable == True:
    layer.trainable = True
  else:
    layer.trainable = False


In [ ]:
# 동결된 conv를 사용해서 모델 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
model2 = Sequential()
model2.add(conv_base) # 전이학습할 모델의 특성추출부 사용하기
model2.add(Flatten())
model2.add(Dense(units = 256, activation = 'relu'))
# 출력층
model2.add(Dense(units = 1, activation = 'sigmoid'))
model2.summary()

In [ ]:
len(model2.trainable_weights)

10

In [ ]:
model2.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy']
               )

In [ ]:
model2.fit_generator(
    generator = train_aug_generator,
    epochs = 20,
    validation_data = val_generator
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/20
20/20 [==============================] - 23s 1s/step - loss: 1.3311 - accuracy: 0.4870 - val_loss: 0.7094 - val_accuracy: 0.4580
Epoch 2/20
20/20 [==============================] - 22s 1s/step - loss: 0.6938 - accuracy: 0.5085 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/20
20/20 [==============================] - 21s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 4/20
20/20 [==============================] - 21s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 5/20
20/20 [==============================] - 21s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/20
20/20 [==============================] - 21s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/20
20/20 [==============================] - 21s 1s/step - loss: 0.6932 - accuracy: 0.4750 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/20
20/20 [==

# 과적합 방지
- dropout : 앞층과 뒷층간의 파라미터수 차이가 큰 곳에 주로 배치 (0.5이하로)
- BatchNormalizer : CNN에서 과적합 방지 과거에는 dropout을 이용해 과적합으 방지했다면 최근에는 CNN과 한쌍으로 BatchNormalizer로 과적합 방지 한다. (정규화), BM의 위치는 정해져있다. Conv2D와 Activation층 사이에 넣어준다. (Activation하면 약간의 정규화하는 기능이 있어서 그전에 넣어주는 것이다)

# CNN

- convolution(특성 추출) : 이미지에 필터를 씌우고 같은 위치끼리 곱하고 결과들을 더해서 하나의 픽셀 값으로 저장, 
- zero padding : 컨볼루션하면 이미지가 축소됨, 이미지 크게 유지하고 싶을 때 씀
- pooling(특성 선택, 이미지 축소) : 주로 maxpooling 씀 
- Flatten : 다차원 배열을 1차원 배열로 변환해주는 것 (Conv1D, RNN은 어짜피 1차원이라 Flatten 쓸필요 없음)

# 전이학습
